# IEEE 802.11n

A etapa de investigação do IEEE 802.11n será dividida em duas etapas, a primeira operando na frequência de 2.4 GHz e a seguinte operando na frequência de 5 GHz

## Requisitos:
* Copiar o exemplo __rate-adaptation-distance.cc__, encontrado em __ns-3.29/examples/wireless__, para a pasta __ns-3.29/scratch__;
* Instalar o gnuplot.

## Alterações:
Para investigar os efeitos dos algoritmos de adaptação de taxa nos diversos padrões IEEE 802.11 serão realizadas algumas alterações no exemplo __rate-adaptation-distance.cc__. A imagem a seguir exibe os principais parâmetros do script que podem ser alterados, resultando em diferentes cenários de análise. 

![parametros](./FIGS/ratedistance.png)

* __staManager__: Define o algoritmo de adaptação utilizado pelo nó STA;
* __apManager__: Define o algoritmo de adaptação utilizado pelo nó AP;
* __standard__: Define qual padrão IEEE 802.11 será utilizado na simulação;
* __OutputFileName__: Define o nome do arquivo de saída da simulação;
* __shortGuardInterval__: Habilita/Desabilita o uso de um Intervalo curto de guarda;
* __chWidth__: Define a largura do canal utilizado na simulação;
* __ap1_x/y__: Define a posição inicial do nó AP;
* __sta1_x/y__: Define a posição inicial do nó STA;
* __steps__: Define a quantidade de passos tomados;
* __stepsSize__: Define a distância tomada em cada passo;
* __stepsTime__: Define o intervalo entre passos.

Neste tutorial, as alterações serão focadas nos algoritmos de adaptação e padrões utilizados. Ao final da simulação é gerado um arquivo *.plt*, usado para plotar o gráfico de throughput vs distância por meio da ferramenta gnuplot.

É importante salientar que o ns-3, através do módulo Wi-Fi, disponibiliza diversos algoritmos de adaptação para construir cenários de simulação variados. Nem todos os algoritmos de adaptação funcionam com qualquer padrão 802.11, assim ficou decidida a utilização do algoritmo de adaptação **minstrel**. Ao final deste tutorial será possível facilmente testar o funcionamento dos outros algoritmos disponíveis no simulador, encontrados no diretório __~/ns-3.29/src/wifi/model__ .

### IEEE 802.11n

O cenário de simulação para este padrão, com frequência de operação em 2.4 GHz, foi configurado do seguinte modo:

![conf2.4](./FIGS/conf24.png)

Perceba que foi utilizado o algoritmo de adaptação __minstrelHt__ ao invés do __minstrel__. Isso acontece devido ao algoritmo minstrel padrão não suportar as altas taxas de transferências (High Throughput - HT) presentes no 802.11n. 

Para a frequência de operação em 5 GHz o cenário de simulação foi configurado do seguinte modo:

![conf5](./FIGS/conf5.png)

Seguindo os passos realizados nas simulações anteriores, obtemos o gráfico de throughput vs distância para o IEEE 802.11n operando nas frequências de 2.4 GHz e 5 GHz.

![throughput5-2-4](./FIGS/throughput-minstrelHt2-4-5.png)

Podemos observar que, utilizando a configuração inicial do script os resultados para as diferentes frequências são semelhantes, comportamento que não é o esperado. Qual seria o motivo desse comportamento?

Na criação do canal de comunicação que é utilizado no cenário de simulação, observamos que ele é criado aplicando parâmetros padrões do modelo de propagação ([__LogDistancePropagationLossModel__](https://www.nsnam.org/doxygen/classns3_1_1_log_distance_propagation_loss_model.html)). 

![yansdefaultcode](./FIGS/yansdefaultcode.png)

A referência de perda padrão para esse modelo de propagação é válida apenas para a frequência de operação 2.4 GHz, quando a frequência de operação for 5 GHz, é necessário então calculá-la por meio do modelo de propagação de Friis com $\lambda = \frac{c}{f}$ e $d = 1$.

$$L_{0} = -10 \cdot log_{10}\left(\frac{\lambda^{2}}{(4\cdot \pi \cdot d)^{2}}\right) = 40.089$$

Assim, no momento em que criamos o canal por meio do *helper*, é necessário fazer a seguinte alteração no código:

![yansaltered](./FIGS/yanschannel_altered.png)

Finalmente, obtemos a comparação real entre o 802.11n operando em 2.4 GHz e 5 GHz.

![802.11n_comparation](./FIGS/throughput-80211n-comparation.png)

Fica evidente a influência da frequência de operação do padrão na sua adaptação de taxa.

### MIMO, Channel Bonding e Short Guard Interval

O padrão IEEE 802.11n possui suporte ao MIMO, Channel Bonding e ao Short Guard Interval. Em respeito ao MIMO, é possivel setar o número de antenas e também fazer com que cada antena transmita um tipo de dado (Spatial Multiplexing).

![MIMO](./FIGS/MIMO_n.png)

Estas opções são configuradas através do [__YansWifiPhyHelper__](https://www.nsnam.org/docs/release/3.29/models/html/wifi-user.html#yanswifiphyhelper).

Já a configuração do *Channel Bonding* é realizada apenas modificando a largura do canal do dispositivo, em que o valor padrão é 20 MHz, para 40 MHz. Em respeito ao *Guard Interval*, o valor padrão, 800 ns, pode ser alterado para 400 ns, que configura um cenário de *Short Guard Interval*. 

![ChannelBonding](./FIGS/ChBonding_n.png)

No script __rate-adaptation-distance.cc__ é possivel facilmente habilitar o *Channel Bonding* e o *Short Guard Interval*, fazendo alterações em dois parâmetros no script.

![habilitar_CB_SGI](./FIGS/hab_CB_SGI.png)

O plot a seguir exibe o comportamento dessas features habilitadas em 2.4 GHz.
![featurecomparsion](./FIGS/ncomparsion.png)

É possivel observar um aumento considerável no throughput do cenário com a utilização das ferramentas disponíveis para esse padrão, chegando a dobrar a taxa de transmissão máxima para o usuário.

O ns-3, através dos exemplos wireless, disponibiliza o script __80211n-mimo.cc__ (_~/ns-3.29/examples/wireless_) criado para validar o MIMO no IEEE 802.11n, plotando o gráfico de throughput vs distância para todos os MCS suportados pelo padrão, utilizando de 1 a 4 antenas. A saída a seguir foi realizada para um cenário com o padrão 802.11n em 5.15 GHz, utilizando o algoritmo de adaptação MinstrelHt.

![plot_mimo](./FIGS/plotn_mimo.png)

### Algoritmos de adaptação suportados

* __IdealWifiManager__;
* __MinstrelHtWifiManager__;
* __ConstantRateWifiManager__;

Com isso, é finalizada a investigação sobre o padrão IEEE 802.11n em ambas frequências de operação suportadas (2.4 GHz e 5 GHz).